# Demonstration notebook for processing raw RINEX data
In this Notebook, we process some example RINEX files to demonstrate gnssvod.

In [1]:
import gnssvod as gv

## gv.preprocess()
The main pre-processing function is preprocess(). This function  will do several things
- It will read RINEX observation files as pandas data frames
- It can aggregate the raw data to a lower temporal rate if specified.
- It will by default download orbit and clock files for the corresponding days from the GSSC ESA server
- From the orbit and clock files, it will calculate azimuth and elevation for each measurement
- It can save each processed file as a netcdf file in the outputdir folder or return the results as a dictionary

### specifying input files
The function exclusively reads RINEX observation files. Such files typically end with the extension '.yyO' where yy is the last two digit of the year. The function can be used to process a single file, a group of files, or several groups of files corresponding to several receivers, as shown in the examples below. All of this is done by specifying a pattern as the first argument to the function.

### specifying output destinations
Results are saved to a NetCDF file when an output directory is specified and/or returned as a dictionary when "outputresult=True" is passed.

Let's read a single file using the example data to begin with

In [8]:
pattern = {'MACROCOSM-2':'data_pr/MACROCOSM-2_raw_202401101542.24O'}
result = gv.preprocess(pattern,outputresult=True)
#here got the same error on integer module zero but I am able to process data below -CVR

data_pr/MACROCOSM-2_raw_202401101542.24O exists | Reading...
Observation file  data_pr/MACROCOSM-2_raw_202401101542.24O  is read in 0.49 seconds.
Processing 24521 individual observations
Calculating Azimuth and Elevation
GFZ0MGXRAP_20240100000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20240100000_01D_05M_ORB.SP3 file is read in 0.42 seconds
GFZ0MGXRAP_20240100000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20240100000_01D_30S_CLK.CLK file is read in 1.42 seconds


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a futur

SP3 interpolation is done in 6.42 seconds


The default logs should indicate how many observations were read in the file. If this is the first time you run the script, it also shows some orbit files were downloaded.

If you process very recent data (less than 3 days old), it could be that the orbit and clock files are not available on the ESA server yet and there would then be an error.

The result returned by the function is a dictionary providing lists of Observation objects.

In [9]:
result

{'MACROCOSM-2': [<gnssvod.io.io.Observation at 0x32f583140>]}

Since we processed one file, there is only one Observation object in the list. Let us access this first and unique item.

In [4]:
obs = result['MACROCOSM-2'][0]
obs

Observation objects are custom classes introduced in the `gnsspy` package by Mustafa Serkan Işık and Volkan Özbey. A significant number of base functions in `gnssvod` are based on gnsspy.

Observation objects contain the following properties
- obs.filename          = the name of the source file
- obs.epoch             = a datetime indicate the day at the start of the record
- obs.observation       = a pandas data frame containing all measurements
- obs.approx_position   = the approximate receiver position as provided in the RINEX file [X,Y,Z]
- obs.receiver_type     = the receiver type if provided in the RINEX file
- obs.antenna_type      = the antenna type if provided in the RINEX file
- obs.interval          = the measurement frequency in seconds
- obs.receiver_clock    = the receiver clock if provided in the RINEX file
- obs.version           = the version of the RINEX file
- obs.observation_types = the observation types reported as columns in obs.observation

Let's just look at the data..

In [7]:
obs.observation

C1C           C1X  C2C           C2I  \
Epoch                      SV                                                   
2024-01-10 14:17:20.199999 G10  2.179389e+07           NaN  NaN           NaN   
                           G26  2.081376e+07           NaN  NaN           NaN   
                           G28  2.173687e+07           NaN  NaN           NaN   
                           G31  2.166509e+07           NaN  NaN           NaN   
                           G32  2.115887e+07           NaN  NaN           NaN   
...                                      ...           ...  ...           ...   
2024-01-10 15:04:57.801999 E25           NaN  2.527410e+07  NaN           NaN   
                           E31           NaN  2.740893e+07  NaN           NaN   
                           C20           NaN           NaN  NaN  2.276216e+07   
                           C32           NaN           NaN  NaN  2.352408e+07   
                           C37           NaN           NaN  NaN  2.398696e+07   

                                         C2X  C7I           C7X       D1C  \
Epoch                      SV                                               
2024-01-10 14:17:20.199999 G10  2.179390e+07  NaN           NaN -4167.363   
                           G26  2.081377e+07  NaN           NaN   746.197   
                           G28  2.173688e+07  NaN           NaN -1773.103   
                           G31  2.166509e+07  NaN           NaN -1046.497   
                           G32  2.115887e+07  NaN           NaN  -826.085   
...                                      ...  ...           ...       ...   
2024-01-10 15:04:57.801999 E25           NaN  NaN  2.527412e+07       NaN   
                           E31           NaN  NaN  2.740894e+07       NaN   
                           C20           NaN  NaN           NaN       NaN   
                           C32           NaN  NaN           NaN       NaN   
                           C37           NaN  NaN           NaN       NaN   

                                     D1X  D2C  ...           L7X   S1C   S1X  \
Epoch                      SV                  ...                             
2024-01-10 14:17:20.199999 G10       NaN  NaN  ...           NaN  49.0   NaN   
                           G26       NaN  NaN  ...           NaN  48.0   NaN   
                           G28       NaN  NaN  ...           NaN  45.0   NaN   
                           G31       NaN  NaN  ...           NaN  47.0   NaN   
                           G32       NaN  NaN  ...           NaN  47.0   NaN   
...                                  ...  ...  ...           ...   ...   ...   
2024-01-10 15:04:57.801999 E25   247.015  NaN  ...  1.017684e+08   NaN  41.0   
                           E31 -3780.169  NaN  ...           NaN   NaN  35.0   
                           C20       NaN  NaN  ...           NaN   NaN   NaN   
                           C32       NaN  NaN  ...           NaN   NaN   NaN   
                           C37       NaN  NaN  ...           NaN   NaN   NaN   

                                S2C   S2I   S2X  S7I   S7X  Azimuth Elevation  
Epoch                      SV                                                  
2024-01-10 14:17:20.199999 G10  NaN   NaN  40.0  NaN   NaN      NaN       NaN  
                           G26  NaN   NaN  41.0  NaN   NaN      NaN       NaN  
                           G28  NaN   NaN  41.0  NaN   NaN      NaN       NaN  
                           G31  NaN   NaN  38.0  NaN   NaN      NaN       NaN  
                           G32  NaN   NaN  39.0  NaN   NaN      NaN       NaN  
...                             ...   ...   ...  ...   ...      ...       ...  
2024-01-10 15:04:57.801999 E25  NaN   NaN   NaN  NaN  36.0      NaN       NaN  
                           E31  NaN   NaN   NaN  NaN  29.0      NaN       NaN  
                           C20  NaN  46.0   NaN  NaN   NaN      NaN       NaN  
                           C32  NaN  46.0   NaN  NaN   NaN      

The pandas data frame has a MultIndex that contains both Epoch and SV as indices. The columns correspond to:
- C# = Pseudorange from the receiver to the satellite, in meters
- L# = Carrier phase, in cycles
- D# = Doppler, in Hz
- S# = Carrier to noise density C/N$_0$, in dB (receiver-dependent)

And the numbers (S1, S2, etc. ) indicate the corresponding GNSS frequency

The azimuth and elevation of the satellite with respect to the receiver are expressed in degrees. Computation speed for the azimuth and elevation can vary according to your hardware. Most of the time is spent interpolating the orbit parameters to the time stamps of each measurement. This is why it is sometimes useful to aggregate high frequency data (here one measurement per second) to for instance one measurement each 15 seconds.

### resampling

We can pass "interval='15S'" to resample the data during the preprocessing. The returned data will be smaller and the calculation of the azimuths and elevations (reported as "SP3 interpolation") will be faster.

In [10]:
pattern = {'MACROCOSM-2':'data_pr/MACROCOSM-2_raw_202401101542.24O'}
result = gv.preprocess(pattern,interval='15S',outputresult=True)
# and show data frame
result['MACROCOSM-2'][0].observation

data_pr/MACROCOSM-2_raw_202401101542.24O exists | Reading...
Observation file  data_pr/MACROCOSM-2_raw_202401101542.24O  is read in 0.46 seconds.
Processing 24521 individual observations


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:200: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  obs.observation = obs.observation[subset].groupby([pd.Grouper(freq=interval, level='Epoch'),pd.Grouper(level='SV')]).mean()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/io/preprocess.py:204: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  obs.interval = pd.Timedelta(interval).seconds


Calculating Azimuth and Elevation
GFZ0MGXRAP_20240100000_01D_05M_ORB.SP3 exists | Reading...
GFZ0MGXRAP_20240100000_01D_05M_ORB.SP3 file is read in 0.28 seconds
GFZ0MGXRAP_20240100000_01D_30S_CLK.CLK exists | Reading...
GFZ0MGXRAP_20240100000_01D_30S_CLK.CLK file is read in 1.51 seconds


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  sp3_temp_resampled = sp3_temp.resample(f"{interval}S")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/gnssvod/position/interpolation.py:48: FutureWarning: 'S' is deprecated and will be removed in a futur

SP3 interpolation is done in 0.86 seconds


C1C  C1X           C2C           C2I  \
Epoch               SV                                                   
2024-01-10 15:43:45 C19           NaN  NaN           NaN  2.556173e+07   
                    G04  2.573829e+07  NaN           NaN           NaN   
                    G16  2.164106e+07  NaN           NaN           NaN   
                    G26  2.129422e+07  NaN           NaN           NaN   
                    G28  2.327215e+07  NaN           NaN           NaN   
...                               ...  ...           ...           ...   
2024-01-10 16:04:30 R07  2.097711e+07  NaN  2.097711e+07           NaN   
                    R08  2.124583e+07  NaN  2.124583e+07           NaN   
                    R09  2.458480e+07  NaN           NaN           NaN   
                    R10  2.418112e+07  NaN           NaN           NaN   
                    R11  2.309832e+07  NaN           NaN           NaN   

                                  C2X  C7I  C7X          D1C  D1X  \
Epoch               SV                                              
2024-01-10 15:43:45 C19           NaN  NaN  NaN          NaN  NaN   
                    G04           NaN  NaN  NaN  1953.582222  NaN   
                    G16           NaN  NaN  NaN  -285.108778  NaN   
                    G26  2.129422e+07  NaN  NaN -2292.311778  NaN   
                    G28  2.327216e+07  NaN  NaN -2002.631111  NaN   
...                               ...  ...  ...          ...  ...   
2024-01-10 16:04:30 R07           NaN  NaN  NaN -3537.544857  NaN   
                    R08           NaN  NaN  NaN   398.604571  NaN   
                    R09           NaN  NaN  NaN -3629.749000  NaN   
                    R10           NaN  NaN  NaN   156.979000  NaN   
                    R11           NaN  NaN  NaN  -937.599714  NaN   

                                 D2C  ...  L7X        S1C S1X        S2C  \
Epoch               SV                ...                                  
2024-01-10 15:43:45 C19          NaN  ...  NaN        NaN NaN        NaN   
                    G04          NaN  ...  NaN  27.666667 NaN        NaN   
                    G16          NaN  ...  NaN  38.000000 NaN        NaN   
                    G26          NaN  ...  NaN  44.777778 NaN        NaN   
                    G28          NaN  ...  NaN  39.222222 NaN        NaN   
...                              ...  ...  ...        ...  ..        ...   
2024-01-10 16:04:30 R07 -2751.401857  ...  NaN  44.000000 NaN  35.000000   
                    R08   310.161857  ...  NaN  41.857143 NaN  31.714286   
                    R09          NaN  ...  NaN  31.285714 NaN        NaN   
                    R10          NaN  ...  NaN  42.000000 NaN        NaN   
                    R11          NaN  ...  NaN  35.857143 NaN        NaN   

                          S2I   S2X  S7I  S7X     Azimuth  Elevation  
Epoch               SV                                                
2024-01-10 15:43:45 C19  38.0   NaN  NaN  NaN  -45.396605  29.187180  
                    G04   NaN   NaN  NaN  NaN  -37.689318   9.279269  
                    G16   NaN   NaN  NaN  NaN -106.934002  60.593220  
                    G26   NaN  20.0  NaN  NaN  -28.286814  67.446587  
                    G28   NaN  38.0  NaN  NaN   41.478383  36.907500  
...                       ...   ...  ...  ...         ...        ...  
2024-01-10 16:04:30 R07   NaN   NaN  NaN  NaN -178.516552  62.175402  
                    R08   NaN   NaN  NaN  NaN  -44.831249  58.265056  
                    R09   NaN   NaN  NaN  NaN   35.314263  13.164595  
                    R10   NaN   NaN  NaN  NaN         NaN        NaN  
                    R11   NaN   NaN  NaN  NaN  -73.712930  30.425331  

[1707 rows x 30 columns]

Orbit and clock files are not downloaded again if they already exist. There are now less rows in the data frame.

## Batch processing
We now use the preprocessing function to process many files and save the outputs as NetCDF files (instead of returning as objects). If we were to process several hundreds of files, the system would likely not have sufficient memory to hold all of the outputs, so it makes sense to processed data as a NetCDF file.

### Specifying several groups of files
Instead of specifying just one file, we use the dictionary to specify a pattern. All files matching the pattern will be processed. We can process several groups files by specifying different matching patterns (see below).

### Specifying where to save data
Same as for specifying the inputs, we use a dictionary to indicate where to save data. The function will create the destination folder if it does not exist.

### Specifying a list of variables to save
For calculating GNSS-VOD, we only need the "S" variables. We can reduce the size of the saved NetCDF files by discarding the other variables, this is done with the 'keepvars' argument, which will only keep the variables present in the passed list. This argument supports UNIX-style pattern matching (e.g. 'S*' will match all variables starting with 'S')

### Compression
Unless `compress=False` is passed as argument, `gv.preprocess()` will compress all S* variables, as well as Azimuth and Elevation when saving to NetCDF. These variables are encoded as Int16 with a scale factor of 0.1. The decoding is automatically applied when reading the data with xarray.

In [12]:
# use gnssvod to batch process the observation RINEX files 
# (files with extension .yyO for each station)
# pattern = {'choice_of_name_for_station1':'pattern to match (UNIX-style)',
#            'choice_of_name_for_station2':'pattern to match (UNIX-style)',
#             ...}
#
pattern = {'MACROCOSM-2':'data_pr/MACROCOSM-2*.24O',}
outputdir = {'MACROCOSM-2':'data_pr/nc/',}
# what variables should be kept
keepvars = ['S1C', 'S1X', 'S2C', 'S2X']

gv.preprocess(pattern,interval='15S',keepvars=keepvars,outputdir=outputdir)

MACROCOSM-2_raw_202401101542.nc already exists, skipping.. (pass overwrite=True to overwrite)


### Skipping existing files by default
The preprocess function will scan the destination folder for existing NetCDF files. If some files are found that have already been processed, these files will be skipped unless overwrite=True has been passed.

Here because the destination folder was empty, a user warning appears in the log above but can be ignored ("Could not find any files matching the pattern data_RINEX2.11/Dav2_Twr/nc/*.nc")